# Grafiken für RLIED am 12.9.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap

In [ ]:
# Pfad zum Ausgeben der Ergebnisse
path = 'output/'

In [ ]:
# Qualität der Grafiken
dpi = 600

In [ ]:
# Laden der Umfrageergebnisse
links = ['E-Mailverteiler', 'internes-Netzwerk', 'Pressemitteilung']
# Funktion zum Bereinigen der Ergebnisse
def cleanse(df):
    mask = (df['duration']>0) \
        & (df['lastpage']>0) \
        & (df['dispcode'].isin([31,32,33,34,22]))
    return df.loc[mask]

# Funktion zum Laden, Filtern und Bereinigen der Daten
def load_df(link, skipped):
    df = pd.read_csv('data/2022_06_RLI_Klimaneutrales Stromsystem_{}_Ergebnisse.csv'.format(link),
                     sep=';', header=2, usecols=list(range(3,123)))
    df = cleanse(df)
    skip_codes = [22] if skipped else [31,32,33,34]
    return df.loc[df['dispcode'].isin(skip_codes)]

# eine Tabelle mit Daten
data = pd.concat([load_df(link, skipped=False) for link in links]).reset_index()
len(data)

In [ ]:
# Importiere Codebook in dict format
from src import rls_umfrage_auswertung
codebook = rls_umfrage_auswertung.main_preprocessing_codebook()
# Corrections
codebook[25]['subquestion'].pop(8)

In [ ]:
# Codes, die Bedeuten, dass keine Angabe gemacht wurde
exclude_codes = [-77, -99, -66, 0, '0', '-66', '-99', '-77']

In [ ]:
# Einstellungen zur Darstellung
# Farbpalette (kann mit eigenen Farben angepasst werden)
cmap = cm.get_cmap('coolwarm')
grey = np.array([11/16, 11/16, 11/16, 1])
light = np.array([.9, .9, .9, 1])
colors5 = ListedColormap(np.vstack([cm.get_cmap('coolwarm', 4)(range(4)), grey]))
colors6 = ListedColormap(np.vstack([cm.get_cmap('coolwarm', 5)(range(5)), grey]))
colors7 = ListedColormap(np.vstack([cm.get_cmap('coolwarm', 6)(range(6)), light, grey]))
colors8 = ListedColormap(np.vstack([cm.get_cmap('coolwarm', 7)(range(7)), light, grey]))
colors8_pie = ListedColormap(np.vstack([cm.get_cmap('coolwarm', 7)(range(7)), grey]))

In [ ]:
# Funktion zum plotten von horizontalen Balken
# returns figure
def plot_bar(code, # Number of the question in codebook
             title=None, # custom title
             question=True, # write questions
             agg={}, # replacement code for reply labels (str) for aggregation
             sort=True, # sort answers by most appreciation
             colors=cmap, # custom colormap
             legend=True, # whether to add a legend under the plot
             legend_cols=4 # number of columns in the legend
            ):
    # Collect data for bars
    bars = {}
    for sub in range(1, len(codebook[code]['subquestion'])+1):
        key = codebook[code]['subquestion'][sub]['columns']
        codes = codebook[code]['subquestion'][sub]['multiple-choice-options']
        q = codebook[code]['subquestion'][sub]['question'] if question else sub
        values = data[key].astype(str).replace(codes)
        values = values.loc[~values.isin(exclude_codes)].replace(agg).value_counts() # aggregate, count
        for val in set(codes.values()).difference(values.index): # fill missing values
            values.loc[val] = 0
        values = values.loc[[label for label in codes.values()]] # sort
        values = values / len(data.loc[~data[key].isin(exclude_codes)]) # get %
        bars[q] = values
    data_to_plot = pd.DataFrame(bars).fillna(0)
        
    # Construct the plot
    # https://stackoverflow.com/questions/41530975/set-size-of-subplot-in-matplotlib
    fig, axis = plt.subplots(figsize=(8, len(bars)*0.5))
    # Sort the plots
    data_to_plot = data_to_plot.sort_values(by=codes["1"], axis=1, ascending=sort)
    ax = data_to_plot.T.plot.barh(
        stacked=True, title=title, mark_right=True, ax=axis, legend=None, cmap=colors)
    #ax.set_aspect(0.05)
    
    # Add a 50% line
    plt.axvline(0.5, color='#555555', ls='--', lw=.5)
    
    # Remove the frame
    for pos in ['right', 'top', 'bottom']:
        plt.gca().spines[pos].set_visible(False)
        
    # Add a legend
    if legend:
        hight = -1.5
        if len(bars) == 3: hight = -0.75
        if len(bars) == 4: hight = -0.6
        if len(bars) == 5: hight = -0.5
        if len(bars) == 6: hight = -0.4
        if len(bars) >= 7: hight = -0.3
        ax.legend(loc='lower center', bbox_to_anchor=(0.5, hight),
                  ncol=legend_cols, fancybox=True, shadow=False)

In [ ]:
# Funktion zum plotten von einem Kuchendiagramm
# returns figure
def plot_pie(code, # Number of the question in codebook
             sub=1, # sub-question
             title=None, # custom title
             agg={}, # replacement code for reply labels (str) for aggregation
             colors=cmap, # custom colormap
             legend=True
            ):
    # Collect data
    key = codebook[code]['subquestion'][sub]['columns']
    codes = codebook[code]['subquestion'][sub]['multiple-choice-options']
    q = codebook[code]['subquestion'][sub]['question']
    values = data[key].astype(str).replace(codes) # to strings
    values = values.loc[~values.isin(exclude_codes)].replace(agg).value_counts() # aggregate, count
    for val in set(codes.values()).difference(values.index): # fill missing values
        values.loc[val] = 0.001
    values = values.loc[[label for label in codes.values()]] # sort
        
    # Construct the plot
    fig, axis = plt.subplots(figsize=(10,4), constrained_layout=False)
    ax = values.plot.pie(ax=axis, cmap=colors, startangle=90, title=title, labeldistance=None)
    ax.axis('off')
    
    # draw a legend
    if legend:
        dist = 1.8
        if len(codes) <= 4: dist = 1.4
        if len(codes) == 5: dist = 2
        if len(codes) == 6: dist = 1.68
        ax.legend(loc='center right', bbox_to_anchor=(dist, 0.5),
                  ncol=1, fancybox=False, frameon=False)

# Auswertung allgemein

In [ ]:
title = 'Hintergrund der Teilnehmende'
plot_pie(40,1,title, colors='Blues')
plt.savefig(path+title+'.png', dpi=dpi)

In [ ]:
title = 'Synonyme KS und ES'
plot_pie(5,1,title)
plt.savefig(path+title+'.png', dpi=dpi)

In [ ]:
title = 'Kompatibilität Klimaziele'
plot_pie(7,1,title)
plt.savefig(path+title+'.png', dpi=dpi)

In [ ]:
title = 'KS als Zwischenschritt zum KE'
plot_pie(9,1,title)
plt.savefig(path+title+'.png', dpi=dpi)

In [ ]:
# ToDo: Mittelwert als gestrichelte Linie
title = 'Jährlicher Strombedarf Deutschlands'
fig, ax = plt.subplots(1,2, figsize=(8,4), sharex='all', sharey='all')
data['v_148'].loc[~data['v_148'].isin(exclude_codes)].hist(bins=15, ax=ax[0], color=cmap(0))
ax[0].set_title('im KS')
ax[0].axvline(data['v_148'].loc[~data['v_148'].isin(exclude_codes)].mean(), color='#555555', ls='--', lw=1.)
data['v_149'].loc[~data['v_149'].isin(exclude_codes)].hist(bins=15, ax=ax[1], color=cmap(0))
ax[1].axvline(data['v_149'].loc[~data['v_149'].isin(exclude_codes)].mean(), color='#555555', ls='--', lw=1.)
ax[1].set_title('im KE')
fig.suptitle(title, fontsize=14)
plt.savefig(path+title+'.png', dpi=dpi)

In [ ]:
title = 'Importbilanz Stromsystem'
plot_bar(13, colors=colors6, legend_cols=2)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Grüner Wasserstoff als entscheidender Baustein'
plot_bar(14, colors=colors5, legend_cols=3, sort=False)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Herkunft grüner Wasserstoff'
plot_pie(15, 1, title, colors=cmap)
plt.savefig(path+title+'.png', dpi=dpi)

# Auswertung Gesellschaft

In [ ]:
title = 'Aufholbedarf Prozessuale Teilhabe'
plot_bar(17, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Aufholbedarf Finanzielle Teilhabe und Geschäftsmodelle'
plot_bar(18, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Suffizienzmaßnahmen zur Nachfragereduktion'
plot_bar(19, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Aufholbedarf Energiegerechtigkeit und Just Transition'
plot_bar(20, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

# Auswertung Wirtschaft

In [ ]:
title = 'Aufholbedarf Bepreisung'
plot_bar(23, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Aufholbedarf Regulatorischer Rahmen und Förderdesign'
plot_bar(24, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Aufholbedarf Stromhandel'
plot_bar(25, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

# Auswertung Technik

In [ ]:
title = 'Aufholbedarf Sektorenkopplung'
plot_bar(28, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Aufholbedarf Digitalisierung'
plot_bar(29, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Aufholbedarf Flexibilitäten'
plot_bar(30, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Aufholbedarf Konventionelle Kraftwerke'
plot_bar(31, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Aufholbedarf Stromnetze'
plot_bar(32, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
title = 'Aufholbedarf Ausbau Erneuerbare'
plot_bar(33, colors=colors7, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

# Auswertung Strommarktdesign

In [ ]:
title = 'Heutiges Strommarktdesign unpassend für Erneuerbare'
plot_pie(38, 2, title, colors=colors8_pie)
plt.savefig(path+title+'.png', dpi=dpi)

In [ ]:
title = 'Reform Strommarktdesign notwendig'
plot_pie(36, 1, title, colors=cmap)
plt.savefig(path+title+'.png', dpi=dpi)

In [ ]:
codebook[38]['subquestion'][1]['question'] = 'Stringentes Marktdesign nötig'
codebook[38]['subquestion'][2]['question'] = 'Heutiges Strommarktdesign unpassend für Erneuerbare'
codebook[38]['subquestion'][3]['question'] = 'Ziel der Strommarktreform ist die koordinierende Funktion'
codebook[38]['subquestion'][4]['question'] = 'Dezentralität wird Strukturmerkmal'
codebook[38]['subquestion'][5]['question'] = 'Vor-Ort-Ebene wird wichtiger Bestandteil'
codebook[38]['subquestion'][6]['question'] = 'Abkehr vom Kupferplatten-Ideal und Anerkennung Engpässe'

In [ ]:
title = 'Strommarktdesign Thesen'
plot_bar(38, question=True, colors=colors8_pie, legend_cols=4)
plt.savefig(path+title+'.png', dpi=dpi, bbox_inches='tight', pad_inches=1)

In [ ]:
rls_umfrage_auswertung.create_wordcloud(codebook, survey_data=data,
                                        question_number_list=[37],
                                        number_of_most_common_words_displayed=20)

In [ ]:
rls_umfrage_auswertung.create_wordcloud(codebook, survey_data=data,
                                        question_number_list=[39],
                                        number_of_most_common_words_displayed=20)

In [ ]:
for subquestion in range(1,7):
    title = 'These '+str(subquestion)
    plot_pie(38,subquestion)
    plt.savefig(path+title+'.png', dpi=dpi)